In [21]:
import source.prepare_data as prd
import source.dataset as dat
import source.multi_layer_perceptron as mlp
import source.table as tb
import source.comparision as comp

In [22]:
import pandas as pd
import spacy
from importlib import reload

In [62]:
reload(prd)
reload(dat)
reload(mlp)
reload(tb)
reload(comp)

<module 'source.comparision' from 'C:\\DL2\\source\\comparision.py'>

In [12]:
"""
1) Preprocessing
"""
# load models and dataset
df = pd.read_csv("hf://datasets/gxb912/large-twitter-tweets-sentiment/train.csv", nrows=100)
nlp = spacy.load("en_core_web_sm")

In [29]:
# prepare dataset
df = prd.prepare_df(df, nlp)

# create dataset out of cleaned columns
dataset = dat.Text_Dataset(df, col_text="clean_text_str", col_label="sentiment", args={"max_features":5000})
dataset.split_dataset()


In [30]:
X_train_TF, X_test_TF, Y_train_TF, Y_test_TF = dataset.get_encodings(tfidf=True)

mlp_tdidf_params = mlp.find_best_mlp(X_train=X_train_TF,
                                     Y_train=Y_train_TF,
                                     X_test=X_test_TF,
                                     Y_test=Y_test_TF,
                                     n_trials=3)

[I 2025-06-13 17:24:56,294] A new study created in memory with name: mlp_optimization
[I 2025-06-13 17:25:08,492] Trial 0 finished with values: [0.6984240412712097, 0.5] and parameters: {'epochs': 139, 'learning_rate': 0.24673574839355933, 'units': 10}.
[I 2025-06-13 17:25:20,765] Trial 1 finished with values: [0.7029567956924438, 0.5] and parameters: {'epochs': 119, 'learning_rate': 0.3677509130523532, 'units': 5}.
[I 2025-06-13 17:25:30,884] Trial 2 finished with values: [0.6917957067489624, 0.550000011920929] and parameters: {'epochs': 100, 'learning_rate': 0.2161590814912232, 'units': 2}.


{'loss': 0.6917957067489624, 'accuracy': 0.550000011920929, 'params_epochs': 100.0, 'params_learning_rate': 0.2161590814912232, 'params_units': 2.0, 'time': np.float64(11.528611)}


In [31]:
vocab_size = comp.test_vocab_size(df)
split_size = comp.test_split_ratio(dataset)

Intializing test of vocab size with: lr = 0.1, epochs = 100, units = 5.
Embeding is BOW.........
The result is: [(1000, 0.55), (2000, 0.5), (3000, 0.55), (4000, 0.4), (5000, 0.55), (6000, 0.55), (7000, 0.6), (8000, 0.65)]
Intializing test split with: lr = 0.1, epochs = 100, units = 5.
Embeding is BOW......
The result is: [(0.1, 0.7), (0.2, 0.6), (0.3, 0.4667), (0.4, 0.55), (0.5, 0.56)]


In [63]:
table = tb.Table()
table.add_record(name="mlp_bow",
                 epoch=mlp_tdidf_params["params_epochs"],
                 lr=mlp_tdidf_params["params_learning_rate"],
                 units=mlp_tdidf_params["params_units"],
                 avg_time=mlp_tdidf_params["time"],
                 split_accuracy=split_size,
                 vocab_accuracy=vocab_size)

Adding a new record with params: name: mlp_bow, epoch: 100.0, lr: 0.2161590814912232, units: 2.0, average time: 11.528611


In [65]:
table.add_record(name="mlp_2",
                 epoch=20,
                 lr=0.5,
                 units=4,
                 avg_time=20.5,
                 split_accuracy=[(0.1, 0.5), (0.2, 0.3), (0.3, 0.4), (0.4, 0.55), (0.5, 0.7)], 
                 vocab_accuracy=[(1000, 0.65), (2000, 0.56), (3000, 0.1), (4000, 0.4), (5000, 0.55), (6000, 0.55), (7000, 0.6), (8000, 0.9)])

Adding a new record with params: name: mlp_2, epoch: 20, lr: 0.5, units: 4, average time: 20.5


In [66]:
table.show(tabulate_view=True)


Table info:
<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            3 non-null      object 
 1   epochs          3 non-null      float64
 2   lr              3 non-null      float64
 3   units           3 non-null      float64
 4   avg_time        3 non-null      float64
 5   split_accuracy  3 non-null      object 
 6   vocab_accuracy  3 non-null      object 
dtypes: float64(4), object(3)
memory usage: 192.0+ bytes
None

Records preview:
| name    |   epochs |       lr |   units |   avg_time | split_accuracy                                                    | vocab_accuracy                                                                                                |
|---------|----------|----------|---------|------------|-------------------------------------------------------------------|----------------------------------------------